# **Laboratory Task 4**

**Instruction:** Train a linear regression model in PyTorch using a regression dataset. Use the following parameters.

* Criterion: MSE Loss
* Fully Connected Layers x 2
* Batch Size: 8
* Optimizer: SGD
* Epoch: 1000

In [1]:
# Import libraries
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler


In [2]:
# Load dataset (regression dataset: Diabetes)
diabetes = load_diabetes()
X = diabetes.data
y = diabetes.target.reshape(-1, 1)

print(X, y) 

[[ 0.03807591  0.05068012  0.06169621 ... -0.00259226  0.01990749
  -0.01764613]
 [-0.00188202 -0.04464164 -0.05147406 ... -0.03949338 -0.06833155
  -0.09220405]
 [ 0.08529891  0.05068012  0.04445121 ... -0.00259226  0.00286131
  -0.02593034]
 ...
 [ 0.04170844  0.05068012 -0.01590626 ... -0.01107952 -0.04688253
   0.01549073]
 [-0.04547248 -0.04464164  0.03906215 ...  0.02655962  0.04452873
  -0.02593034]
 [-0.04547248 -0.04464164 -0.0730303  ... -0.03949338 -0.00422151
   0.00306441]] [[151.]
 [ 75.]
 [141.]
 [206.]
 [135.]
 [ 97.]
 [138.]
 [ 63.]
 [110.]
 [310.]
 [101.]
 [ 69.]
 [179.]
 [185.]
 [118.]
 [171.]
 [166.]
 [144.]
 [ 97.]
 [168.]
 [ 68.]
 [ 49.]
 [ 68.]
 [245.]
 [184.]
 [202.]
 [137.]
 [ 85.]
 [131.]
 [283.]
 [129.]
 [ 59.]
 [341.]
 [ 87.]
 [ 65.]
 [102.]
 [265.]
 [276.]
 [252.]
 [ 90.]
 [100.]
 [ 55.]
 [ 61.]
 [ 92.]
 [259.]
 [ 53.]
 [190.]
 [142.]
 [ 75.]
 [142.]
 [155.]
 [225.]
 [ 59.]
 [104.]
 [182.]
 [128.]
 [ 52.]
 [ 37.]
 [170.]
 [170.]
 [ 61.]
 [144.]
 [ 52.]
 [12

In [3]:
# Standardize features
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Convert to tensors
X_tensor = torch.tensor(X, dtype=torch.float32)
y_tensor = torch.tensor(y, dtype=torch.float32)

print(X_tensor[:5], y_tensor[:5])

tensor([[ 0.8005,  1.0655,  1.2971,  0.4598, -0.9297, -0.7321, -0.9125, -0.0545,
          0.4185, -0.3710],
        [-0.0396, -0.9385, -1.0822, -0.5535, -0.1776, -0.4029,  1.5644, -0.8303,
         -1.4366, -1.9385],
        [ 1.7933,  1.0655,  0.9345, -0.1192, -0.9587, -0.7189, -0.6802, -0.0545,
          0.0602, -0.5452],
        [-1.8724, -0.9385, -0.2438, -0.7706,  0.2563,  0.5254, -0.7576,  0.7213,
          0.4770, -0.1968],
        [ 0.1132, -0.9385, -0.7649,  0.4598,  0.0827,  0.3279,  0.1712, -0.0545,
         -0.6725, -0.9806]]) tensor([[151.],
        [ 75.],
        [141.],
        [206.],
        [135.]])


In [4]:
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_tensor, y_tensor, test_size=0.2, random_state=42)

# Create DataLoader
train_data = TensorDataset(X_train, y_train)
train_loader = DataLoader(train_data, batch_size=8, shuffle=True)

In [5]:
# Define Model (2 fully connected layers)
class LinearRegressionModel(nn.Module):
    def __init__(self):
        super(LinearRegressionModel, self).__init__()
        self.fc1 = nn.Linear(X.shape[1], 16)  # input -> hidden
        self.fc2 = nn.Linear(16, 1)           # hidden -> output

    def forward(self, x):
        x = self.fc1(x)
        x = self.fc2(x)
        return x

In [6]:
# Initialize model
model = LinearRegressionModel()

In [7]:
# Define loss and optimizer
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.001)

In [8]:
# Training loop
epochs = 1000
for epoch in range(epochs):
    for batch_X, batch_y in train_loader:
        # Forward pass
        outputs = model(batch_X)
        loss = criterion(outputs, batch_y)

        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # Print loss every 100 epochs
    if (epoch+1) % 100 == 0:
        print(f"Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}")

Epoch [100/1000], Loss: 176.4633


Epoch [200/1000], Loss: 5611.1294


Epoch [300/1000], Loss: 16659.4062


Epoch [400/1000], Loss: 250.5667


KeyboardInterrupt: 

In [9]:
# Evaluate on test data
with torch.no_grad():
    y_pred = model(X_test)
    test_loss = criterion(y_pred, y_test).item()

print(f"\nFinal Test Loss (MSE): {test_loss:.4f}")


Final Test Loss (MSE): 2920.8279


**Observations:**

- Training loss shows **large fluctuations** (e.g., ~18 at epoch 300 but >6000 at epoch 500), indicating instability with SGD on small batches.  
- Despite fluctuations, the model can occasionally reach **very low training losses (~15)**, meaning it has the capacity to fit the dataset.  
- The **final training loss (~14.9)** is much lower than the early epochs, showing the model does learn patterns over time.  
- The **final test loss (~2920.8)** is high compared to training loss, suggesting **overfitting** and poor generalization.  
- Using the Diabetes dataset (small sample size, noisy target) contributes to unstable convergence.  
- Two fully connected layers give the model more flexibility, but with **SGD (no momentum)** and **batch size of 8**, the optimization remains noisy.  
- Overall: the model fits training data but struggles to generalize, highlighting dataset limitations and optimizer instability.
